In [2]:
using PyPlot
using PyCall
numpy = pyimport("numpy")
#import Pkg; Pkg.add("Numpy")

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.


┌ Info: Installing pyqt package to avoid buggy tkagg backend.
└ @ PyPlot /Users/sakurairihito/.julia/packages/PyPlot/XaELc/src/init.jl:118
┌ Info: Installing PyQt5 via the Conda pyqt package...
└ @ PyCall /Users/sakurairihito/.julia/packages/PyCall/L0fLP/src/PyCall.jl:711
┌ Info: Running `conda install -y pyqt` in root environment
└ @ Conda /Users/sakurairihito/.julia/packages/Conda/1403Y/src/Conda.jl:129

PackagesNotFoundError: The following packages are not available from current channels:

  - pyqt

Current channels:

  - https://conda.anaconda.org/conda-forge/osx-arm64
  - https://conda.anaconda.org/conda-forge/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




PyObject <module 'numpy' from '/Users/sakurairihito/.julia/conda/3/lib/python3.9/site-packages/numpy/__init__.py'>

In [3]:
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["text.usetex"] = true
rcParams["font.family"] = "serif"
rcParams["font.size"] = 16
rcParams["text.latex.preamble"] = raw"\usepackage{amsmath}"

"\\usepackage{amsmath}"

In [4]:
import Pkg; Pkg.add("OMEinsum")
import Pkg; Pkg.add("SymEngine")
import SciPy
using OMEinsum, SymEngine

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [5]:
import Pkg; Pkg.add("FastGaussQuadrature")
using FastGaussQuadrature

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [15]:
function rho_omega(omega)
    return (1/(2*pi)) * sqrt.(4 .- omega.^2)
end

#test ok
a = [-2,2]
println(a.^2)
println(rho_omega(a))

[4, 4]
[0.0, 0.0]


In [121]:
using SparseIR
D = 2.0
temp = 0.1/D
beta = 1 / temp
wmax = 10.0 * D
lambda = beta * wmax
eps = 1e-9

wmax = lambda / beta
k = KernelFFlat(lambda)
basis = FiniteTempBasis(k, fermion, beta, eps)
L = basis.size

basis.beta

LoadError: type FiniteTempBasis has no field beta

スパースサンプリング

In [8]:
samp_taus = TauSampling(basis)
n_taus = size(samp_taus.sampling_points) # (40,)
#println(samp_taus.sampling_points)
n_taus = n_taus[1]
#println("# of sampling points in tau=", n_taus)
#  self.cond = self.matrix.s[0] / self.matrix.s[-1]
condition_num = samp_taus.cond
#println("cond taus=", condition_num)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /Users/sakurairihito/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


11.408397315310255

松原周波数でのサンプリングポインt

In [9]:
samp_matsu = MatsubaraSampling(basis)
n_matsu = size(samp_matsu.sampling_points)[1]
println(n_matsu)
#println(samp_matsu.sampling_points)
#n_matsu = n_matsu[1]
#println(n_matsu)
#println(n_matsu)
condition_matsu = samp_matsu.cond
#println("condition_matsu=" , condition_matsu)



44


17.755472693820177

In [10]:
function scale_quad(x::Vector{Float64}, w::Vector{Float64}, xmin::Float64, xmax::Float64)
    """
    Scale weights and note of quadrature to the interval [xmin, xmax]
    """
        @assert xmin < xmax
        dx = xmax - xmin
        w_ = 0.5 * dx * w
        x_ = (0.5 * dx) * (x .+ 1) .+ xmin
        return x_, w_
end

# test
x = [0.0, 1.0]
w = [0.0, 0.0]
x_min = 0.0
x_max = 1.0

res, res2 = scale_quad(x, w, x_min, x_max)
print(res)

[0.5, 1.0]

In [106]:
#Module iptsolver
struct IPTSolver
    U::Float64
    basis
    beta::Float64
    rho::Function
    quad_weight::Vector{Float64}
    quad_omega::Vector{Float64}
    samp_matsu
    samp_tau

    function IPTSolver(U::Float64, basis, rho::Function,
        omega_min::Float64, omega_max::Float64, deg_leggaus::Int64=1000)
        @assert omega_min < omega_max
        
        beta = 1 / temp
        x_, w_ = gausslegendre(deg_leggaus)
        quad_omega, quad_weight = scale_quad(x_, w_, omega_min, omega_max)
        samp_matsu = MatsubaraSampling(basis)
        samp_tau = TauSampling(basis)
        new(
            U,
            basis,
            beta,
            rho,
            quad_weight,
            quad_omega,
            samp_matsu,
            samp_tau
        )
    end
end

#end

U = 2.0
typeof(rho_omega)
solver.beta

20.0

In [112]:
function make_gloc_iv(solver::IPTSolver, sigma_iv::Vector{Complex{Float64}})

    """
    Compute G_loc(iv) from Sigma_iv
    G_loc(iv) \\simeq \\sum_q w_q rho(x_q)/(iv - w_q - \\Sigma(iv))
    """

    iv = 1im .* solver.samp_matsu.sampling_points .* π / solver.beta 
    #gloc_iv = Vector{Complex{64}}(undef, length(iv)) #undefは空っぽの配列（型は指定しない）
    #for iq in 1:length(solver.quad_weight)
    #end
    newaxis = [CartesianIndex()]

    #println("solver.quad_weight", typeof(solver.quad_weight))
    #println("solver.rho(solver.quad_weight)", typeof(solver.rho(solver.quad_weight))) 
    r = ein"q,q,wq->w"(
            solver.quad_weight,
            solver.rho(solver.quad_omega),
            1 ./ ((iv .- sigma_iv)[:, newaxis] .- solver.quad_omega[newaxis, :])
        )
    return r
end

make_gloc_iv (generic function with 1 method)

In [113]:
u = 1.0
omega_min = -2.0
omega_max = 2.0
sigma_iv_ = make_sigma_ini(solver)
gloc_iv = make_gloc_iv(solver, sigma_iv_)
# normalize_factor(u, basis, rho_omega, omega_min, omega_max,sigma_iv_)
# normalize_factor

44-element Vector{ComplexF64}:
    8.95445166827226e-21 + 0.007242165900471782im
   4.369286741159433e-22 + 0.007605531039445112im
  -3.165925954105526e-21 + 0.008007282450606576im
   1.567064589913301e-20 + 0.018772726200149543im
   7.380478948943469e-20 + 0.03164091653450508im
  -5.836001559631694e-20 + 0.04636868650500979im
  -3.221947714287089e-19 + 0.06278320680890101im
   -7.35621438115605e-19 + 0.0842797091056929im
   4.937121060829541e-19 + 0.10667381086318391im
   4.776639233200245e-19 + 0.13874761525382404im
  4.1233884404826705e-18 + 0.17624161645084063im
 -1.7099986338772908e-18 + 0.20985629372161085im
  2.0789654412387128e-18 + 0.2583210005252233im
                         ⋮
 -1.7099986338772908e-18 - 0.20985629372161085im
  4.1233884404826705e-18 - 0.17624161645084063im
   4.776639233200245e-19 - 0.13874761525382404im
   4.937121060829541e-19 - 0.10667381086318391im
   -7.35621438115605e-19 - 0.0842797091056929im
  -3.221947714287089e-19 - 0.06278320680890101im
  -5.83600

    # r = numpy.einsum("q,q,wq->w",
    #         solver.quad_weight,
    #         solver.rho(solver.quad_omega),
    #         1/((iv .- sigma_iv)[:, newaxis] .- solver.quad_omega[newaxis, :]),
    #         optimize = True
    #     )

r = ein"q,q,wq->w"(
            solver.quad_weight,
            solver.rho(solver.quad_weight),
            1/((iv-sigma_iv)[:,:] .- solver.quad_omega[:,:])
        )
    return r

In [98]:
U = 2.0
#typeof(rho_omega)
solver = IPTSolver(U, basis, rho_omega, -2.0, 2.0)
#println("solver.quad_weight",solver.quad_weight)
println("rho(solver.quad_weight)", size(rho_omega(solver.quad_weight)))
println("solver.quad_omega=", typeof(solver.quad_omega))
println("solver.quad_omega=", typeof(solver.quad_omega[:,:]))
println("solver.quad_omega=", size(solver.quad_omega[:,:]))
#sigma_iv_ = make_sigma_ini(solver)
#new_sigma_iv = new_sigma_iv(solver, sigma_iv_)

iv = 1im * solver.samp_matsu.sampling_points * π/solver.beta 





rho(solver.quad_weight)(1000,)
solver.quad_omega=Vector{Float64}
solver.quad_omega=Matrix{Float64}
solver.quad_omega=(1000, 1)


44-element Vector{ComplexF64}:
    8.95445166827226e-21 + 0.007242165900471782im
   4.369286741159433e-22 + 0.007605531039445112im
  -3.165925954105526e-21 + 0.008007282450606576im
   1.567064589913301e-20 + 0.018772726200149543im
   7.380478948943469e-20 + 0.03164091653450508im
  -5.836001559631694e-20 + 0.04636868650500979im
  -3.221947714287089e-19 + 0.06278320680890101im
   -7.35621438115605e-19 + 0.0842797091056929im
   4.937121060829541e-19 + 0.10667381086318391im
   4.776639233200245e-19 + 0.13874761525382404im
  4.1233884404826705e-18 + 0.17624161645084063im
 -1.7099986338772908e-18 + 0.20985629372161085im
  2.0789654412387128e-18 + 0.2583210005252233im
                         ⋮
 -1.7099986338772908e-18 - 0.20985629372161085im
  4.1233884404826705e-18 - 0.17624161645084063im
   4.776639233200245e-19 - 0.13874761525382404im
   4.937121060829541e-19 - 0.10667381086318391im
   -7.35621438115605e-19 - 0.0842797091056929im
  -3.221947714287089e-19 - 0.06278320680890101im
  -5.83600

In [115]:
function g0_iv(gloc, sigma_iv::Vector{Complex{Float64}}) # sigma_ivは適当な初期値が入る（）
    return 1 ./ (1 ./gloc .+ sigma_iv)
end

g0_iv (generic function with 2 methods)

In [118]:

a = g0_iv(gloc_iv,sigma_iv_);

In [100]:
function sigma_iv(solver::IPTSolver, g0_iv) #g0_ivは上で定義した関数の値が入る
    """
    Compute Sigma(tau) from \\mathcal{G}(iv)
    """
    g0_IR = solver.samp_matsu.fit(g0_iv, axis=1)
    g0_tau = solver.samp_tau.evaluate(g0_IR)
    sigma_tau = (solver.U)^2 .* (g0_tau).^2 .* g0_tau[end:-1:1]
    sigma_IR = solver.samp_tau.fit(sigma_tau)
    return solver.samp_matsu.evaluate(sigma_IR, axis=1)
end

sigma_iv (generic function with 1 method)

In [120]:
sigma_iv(solver, a)

LoadError: type MatsubaraSampling has no field fit

In [101]:
function new_sigma_iv(solver::IPTSolver, sigma_iv)
    gloc_iv = make_gloc_iv(solver, sigma_iv)
    g0_iv = g0_iv(gloc_iv, sigma_iv)
    r = sigma_iv(solver, g0_iv)
    return r
end


new_sigma_iv (generic function with 1 method)

In [102]:
sigma_ini = Vector{Complex{Float64}}
sigma_ini = zeros(ComplexF64, )
#n_matsu = size(samp_matsu.sampling_points)[1]

function make_sigma_ini(solver::IPTSolver)
    samp_matsu = solver.samp_matsu
    n_matsu = size(samp_matsu.sampling_points)[1]
    return zeros(ComplexF64, n_matsu)
end

make_sigma_ini (generic function with 1 method)

In [103]:
#sigma_ini = make_sigma_ini(solver);


In [104]:
niter = 10000
mixing = 0.1

U = 2.0 .* [range(0.001/D, 6.5/D , length=8);] 

#solver = IPTSolver(u, basis, rho_omega, omega_min, omega_max)


function normalize_factor(u, basis, rho_omega, omega_min::Float64, omega_max::Float64, sigma_iv::Vector{Complex{Float64}})
    solver = IPTSolver(u, basis, rho_omega, omega_min, omega_max)

    #if sigma_ini == "None"
    #   sigma_ini = [0] #sigma_iv::Vector{Complex{Float64}})
    #end
    #sigma_ini = Vector
    #sigma_iv = copy(sigma_iv)

    new_sigma_iv = make_sigma_ini(solver)

    for iter in range(1,10)
        new_sigma_iv[1] = new_sigma_iv(solver, sigma_iv)[1]
        rerr = findmax(abs.(new_sigma_iv - sigma_iv))[1]/findmax(abs.(sigma_iv))[1]
        if rerr < 1e-5
            break
        end
        sigma_iv = mixing * new_sigma_iv .+ (1 - mixing) * sigma_iv
    end

    center = length(sigma_iv)÷2 
    z = 1/(1-(imag.(sigma_iv)[center]-imag(sigma_iv)[center-1])/(solver.samp_matsu.sampling_points[center]-solver.samp_matsu.sampling_points[center-1]))

    return z, sigma_iv
end

normalize_factor (generic function with 1 method)

LoadError: MethodError: no method matching /(::Int64, ::Matrix{ComplexF64})
[0mClosest candidates are:
[0m  /(::Any, [91m::ChainRulesCore.AbstractThunk[39m) at ~/.julia/packages/ChainRulesCore/uxrij/src/tangent_types/thunks.jl:35
[0m  /(::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}, [91m::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:93
[0m  /(::Union{Integer, Complex{<:Union{Integer, Rational}}}, [91m::Rational[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/rational.jl:345
[0m  ...

In [ ]:
Z = []
Simga_U = []
for u in U:
    zz, s_u = normalize_factor(u, basis, rho_omega, omega_min, omega_max,sigma_iv)

ErrorException: syntax: line break in ":" expression

In [95]:
length(sigma_ini)
length(sigma_ini)÷2

solver.quad_weightVector{Float64}
solver.rho(solver.quad_weight)Vector{Float64}


0

プレイグランド

In [102]:
x_, w_ = gausslegendre(deg_leggaus=1000)
quad_omega, quad_weight = scale_quad(x_, w_, omega_min, omega_max)

MethodError: MethodError: no method matching gausslegendre(; deg_leggaus=1000)
Closest candidates are:
  gausslegendre(!Matched::Integer) at ~/.julia/packages/FastGaussQuadrature/BRLTf/src/gausslegendre.jl:22 got unsupported keyword argument "deg_leggaus"

using FastGaussQuadrature,SpecialPolynomials

In [103]:
Pkg.add("SpecialPolynomials")
using FastGaussQuadrature,SpecialPolynomials

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [104]:
# ガウスの積分公式を使っている。不定積分を解析的に求めるのではなく、有限な数のサンプル点における値を利用する。
# http://nkl.cc.u-tokyo.ac.jp/12e/02-FEM/GaussLegendre.pdf
x, w = gausslegendre(3)

([-0.7745966692414834, 0.0, 0.7745966692414834], [0.5555555555555556, 0.8888888888888888, 0.5555555555555556])

In [105]:
using LinearAlgebra
myfunc(x) = x^4
I = dot(w,myfunc.(x))
print("x",x)
print
println(I)
@assert I ≈ 2/5


0.4000000000000001


In [106]:
p3 = SpecialPolynomials.basis(Legendre, 3)
x2 = collect(range(-1,1,length=1000))

plt.figure()
plt.plot(x2,p3.(x2),label="P_3")
plt.plot(x,[0,0,0],marker="o")
plt.xlabel("x")
plt.ylabel("3rd basis of Legendre polynomials")
plt.savefig("check_gaussquad.pdf")

In [107]:
x2 = collect(range(-1,1,length=1000))

1000-element Vector{Float64}:
 -1.0
 -0.997997997997998
 -0.995995995995996
 -0.993993993993994
 -0.991991991991992
 -0.98998998998999
 -0.987987987987988
 -0.985985985985986
 -0.983983983983984
 -0.9819819819819819
  ⋮
  0.983983983983984
  0.985985985985986
  0.987987987987988
  0.98998998998999
  0.991991991991992
  0.993993993993994
  0.995995995995996
  0.997997997997998
  1.0

In [108]:
range(-1,1,length=1000)

-1.0:0.002002002002002002:1.0

In [109]:
A = rand(Float64, 2, 2)

2×2 Matrix{Float64}:
 0.284444  0.389023
 0.381043  0.223978

In [111]:
A[end:-1:1, :]

2×2 Matrix{Float64}:
 0.381043  0.223978
 0.284444  0.389023

In [112]:
A[1:2, :]

2×2 Matrix{Float64}:
 0.284444  0.389023
 0.381043  0.223978

In [113]:
A[end:-1:1, :, :]

2×2×1 Array{Float64, 3}:
[:, :, 1] =
 0.381043  0.223978
 0.284444  0.389023

In [114]:
B = rand(Float64, 3, 3)
#println(B)

3×3 Matrix{Float64}:
 0.838049  0.249068  0.970057
 0.637646  0.366873  0.908885
 0.798598  0.96509   0.843229

In [115]:
# 列を交換した。全ての列に関して逆順にした。
B[:, end:-1:1]

3×3 Matrix{Float64}:
 0.970057  0.249068  0.838049
 0.908885  0.366873  0.637646
 0.843229  0.96509   0.798598

In [116]:
B[end:-1:1, :]

3×3 Matrix{Float64}:
 0.798598  0.96509   0.843229
 0.637646  0.366873  0.908885
 0.838049  0.249068  0.970057

In [117]:
C = rand(Float64, 4, 4)

4×4 Matrix{Float64}:
 0.288775  0.193691  0.482939  0.0434422
 0.676368  0.530307  0.383387  0.611905
 0.688957  0.230515  0.680132  0.309188
 0.385985  0.747789  0.631511  0.44173

In [118]:
C[end:-1:1, :]

4×4 Matrix{Float64}:
 0.385985  0.747789  0.631511  0.44173
 0.688957  0.230515  0.680132  0.309188
 0.676368  0.530307  0.383387  0.611905
 0.288775  0.193691  0.482939  0.0434422

In [119]:
C[:,None]

UndefVarError: UndefVarError: None not defined

In [120]:
A = Array{Int}(undef,2,4,2)

2×4×2 Array{Int64, 3}:
[:, :, 1] =
 3417225421390113141  0  0  0
        293168366641  0  0  0

[:, :, 2] =
 0  0  0  0
 0  0  0  0

In [121]:
A[..,1] 

UndefVarError: UndefVarError: .. not defined

In [122]:
A = Array{Int}(undef,3,3)

3×3 Matrix{Int64}:
 7598251388580490081  7022329182242039649  7593476290060186227
 8460625631237138792  8746397786379086196  3345612038820754292
 7738135719211788652  3407352825314488179           4294995827

In [123]:
A[..,1]

UndefVarError: UndefVarError: .. not defined

In [124]:
A = rand(3)

3-element Vector{Float64}:
 0.2633701009362738
 0.35134549825135764
 0.9286167114491054

In [125]:
A[:,:] #:で範囲外の次元を増やしてくれる。あるいは、ある次元に沿った要素を全て指定してくれる、

3×1 Matrix{Float64}:
 0.2633701009362738
 0.35134549825135764
 0.9286167114491054

In [126]:
#juliaで，aからbまで等間隔にp個区切りたかったら，
a = 1
b = 2
p = 10
c = [ range(a,b,length=p); ]
println(c)
d = collect(range(a, b, length=p))
println(d)
length(d)


[1.0, 1.1111111111111112, 1.2222222222222223, 1.3333333333333333, 1.4444444444444444, 1.5555555555555556, 1.6666666666666667, 1.7777777777777777, 1.8888888888888888, 2.0]
[1.0, 1.1111111111111112, 1.2222222222222223, 1.3333333333333333, 1.4444444444444444, 1.5555555555555556, 1.6666666666666667, 1.7777777777777777, 1.8888888888888888, 2.0]


10

In [127]:
e = collect(1.0:1.0:10)

10-element Vector{Float64}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0

In [128]:
function f(a, s="None")
    if s == "None"
        println(a)
    end
end

f(1)


1


In [129]:
using OMEinsum
catty = fill(Basic(:🐱), 2, 2)
fish = fill(Basic(:🐟), 2, 3, 2)
snake = fill(Basic(:🐍), 3, 3)
medicine = ein"ij,jki,kk->k"(catty, fish, snake)


3-element Vector{Basic}:
 4*🐱*🐍*🐟
 4*🐱*🐍*🐟
 4*🐱*🐍*🐟

In [130]:
complex(0)

0 + 0im

In [131]:
for iter in range(1,10)
    println(iter)
end


1
2
3
4
5
6
7
8
9
10


In [132]:
ComplexF64()

MethodError: MethodError: no method matching ComplexF64()
Closest candidates are:
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/char.jl:50
  (::Type{T})(!Matched::Base.TwicePrecision) where T<:Number at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/twiceprecision.jl:255
  Complex{T}(!Matched::Complex) where T<:Real at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/complex.jl:43
  ...

In [133]:
Array{ComplexF64}(42)

MethodError: MethodError: no method matching (Array{ComplexF64})(::Int64)
Closest candidates are:
  (Array{T})(!Matched::LinearAlgebra.LQPackedQ{T, S} where S<:AbstractMatrix{T}) where T at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/lq.jl:168
  (Array{T})(!Matched::UniformScaling, !Matched::Tuple{Int64, Int64}) where T at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:494
  (Array{T})(!Matched::UniformScaling, !Matched::Integer, !Matched::Integer) where T at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:495
  ...

[0.5, 1.0]

In [22]:
a = [1,2,3]
b = [1,2,3,4,5]
println(a[:,:])
println(b[:,:])


[1; 2; 3;;]
[1; 2; 3; 4; 5;;]


In [25]:
# https://compneuro-julia.github.io/appendix/tips.html

v = rand(3)


3-element Vector{Float64}:
 0.48102117655001453
 0.26779376324341264
 0.7763749611208386

In [26]:
newaxis = [CartesianIndex()]
v1 = v[newaxis, :]

1×3 Matrix{Float64}:
 0.481021  0.267794  0.776375

In [27]:
v2 = v[:, newaxis]

3×1 Matrix{Float64}:
 0.48102117655001453
 0.26779376324341264
 0.7763749611208386

In [28]:
v2 .- v1

3×3 Matrix{Float64}:
  0.0       0.213227  -0.295354
 -0.213227  0.0       -0.508581
  0.295354  0.508581   0.0

In [ ]:
v = [1,2,3]
w = [1,2,3,4,5]
newaxis = [CartesianIndex()]
v_ = v[newaxis, :]
println("")
w_ = w[:, newaxis]

w_ .- v_

#複素数
a = [1+im*1, 2+2*im] #3*1行列
b = [3+im*1, 2+2*im, 3+3*im] #1*3行列

c = [1+1*im, 2+2*im, 3+3*im]
d = [1+1*im, 2+2*im, 3+3*im]
println("c=", c)
println("typeof c=", typeof(c))
println("d=", d)
println("typeof d=", typeof(d))

e = a[:,newaxis] .- b[newaxis, :]
println("e=", e)
println("typeof e=", typeof(e))

r = ein"q,q,wq->w"(c,d,e)
#
print(r)

In [ ]:
#println("v", size(v[:,:]))

#newaxis
newaxis = [CartesianIndex()]
#println("v_newaxis=", v[:,newaxis])

#println("solver.quad_omega[newaxis,:]=", solver.quad_omega[newaxis,:])

a = v[:,newaxis] .- solver.quad_omega[newaxis,:]
sigma_iv_ = make_sigma_ini(solver)
b = (iv .- sigma_iv_)[:, newaxis]
c = b .- solver.quad_omega[newaxis,:]
d = rho_omega(solver.quad_omega)
e = solver.quad_weight

f = 1 ./ (c)

#r = ein"q,q"



r = ein"q,q,wq->w"(
            e,
            d,
            f
        )


r_ = ein"q,q,wq->w"(
        solver.quad_weight,
        rho_omega(solver.quad_omega),
        1 ./ ((iv .- sigma_iv_)[:, newaxis] .- solver.quad_omega[newaxis, :])
    )